###**Model calculation: Interferometer**



> This code can be used to reproduce the data plotted in Figure 2 of Schultz et al., Coherence in chemistry: foundations and frontiers. *Chem. Revs.* (submitted)

To begin, press the play button beside each cell (proceed sequentially through the cells).

In [ ]:
#@title Import libraries, prep plot settings

import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

def gensignal(N, trange, Autcrr):
    alpha_real = np.random.normal(0, 1/4, int(N))
    alpha_im = np.random.normal(0,1/4, int(N))

    #initialize empty complex array to store random gaussian samples
    alpha = np.zeros(N+1)
    alpha = alpha.astype(complex)

    for mu in range(1, int(N/2)+1):
        real = alpha_real[mu]
        im = 1j*alpha_im[mu]
        alpha[mu] = real + im
        alpha[-mu] = np.conjugate(alpha[mu])

    # create alpha for zero freq component
    alpha[0] = 1

    t= np.linspace(0, trange, N+1)
    #construct the spectral representation of the signal A using the random samples alpha_w
    fourtransAutcrr = np.fft.fft(Autcrr)
    A_w = [alpha[mu]*fourtransAutcrr[mu] for mu in range(-int(N/2), int(N/2)+1)]




    A_w = np.fft.ifftshift(A_w)

    A_t = np.fft.ifft(A_w)
    return np.real(A_t) #discard small imaginary part

#calculates the autocorrelation function signal[i]*signal[i+j] averaged over Neave different i's
def autocorrelation(signal, j, Neave):
    sum = 0
    signalj = np.roll(signal, -j)       #creates a copy of the timeseries and shifts it over by j
    signal = signal[:Neave]                #remove data from lists that are not included in the ergotic average (Neave is number of timestamps in ergotic average)
    signalj= signalj[:Neave]
    return np.dot(signal, signalj)/Neave    #returns the ergotic average of signal[i]*signal[i+j] over Neave different i's

In [ ]:
#@title Tunable parameters

N=2**20
dt = 0.01
trange = (N+1)*dt
t= np.linspace(0, trange, N+1)

epsilon = 20
tau =   20
period = 1
omega = 2*math.pi/period

In [ ]:
#@title Calculate and plot


###############
## Calculate main pulse
###############
Autcrr = [np.cos(omega*t)*epsilon/tau*np.exp(-t/tau) for t in t]
sig = gensignal(N, trange, Autcrr)


## Set up delay vectors
###################
Ndelaysamples = 150 #number of data points in delay plot
tdelayrange = 8*period
dtdelay = tdelayrange/Ndelaysamples
dN = int(dtdelay/dt)
t_delay = np.linspace(0, tdelayrange, Ndelaysamples)


integrationtime = 2000 * tau
Ninteg = int(integrationtime/dt)


#Calculate the integrated power on the detector sig[i]sig[i+j] where j is the delay and i sums over the integration time
##########
detectorsig = np.zeros(Ndelaysamples)

for delay in range(0, Ndelaysamples):
    sigdelay = np.roll(sig, -delay*dN)
    sigdelaydetect = sigdelay[:Ninteg]
    sigdetect = sig[:Ninteg]
    intensitysum = [(sigdelaydetect[j]+sigdetect[j])**2 for j in range(len(sigdetect))] #instantaneous intensity at time j recorded over the integration time
    detectorsig[delay] = np.mean(intensitysum) #average intensity


intensitytest = [sig[j]**2 for j in range(len(sig))]
val = 2*np.mean(intensitytest)
incoherentsum = [val for j in range(Ndelaysamples)]


############
## Plotting
############

fig = plt.figure(figsize = (3,6))

ax1 = fig.add_subplot(3,1,1)
ax1.plot(t, sig, linewidth = 1, color = 'b', label = 'Pulse')
ax1.plot(t, np.roll(sig, int(2.4*period/dt)), linewidth = 1, color = 'g', label = 'Delay') #comparison between reference and sample delay
plt.xlim(0, 7*period)
ax1.axes.xaxis.set_ticks([])
ax1.axes.yaxis.set_ticks([])
ax1.set_xlabel('Time')
ax1.set_ylabel('Signal')
ax1.set_title('Pulse Profiles')
ax1.legend(prop={'size': 7})

ax2 = fig.add_subplot(3,1,2)
ax2.plot(t_delay, detectorsig, linewidth = 1, color = 'black', label = 'Interference')
ax2.plot(t_delay, incoherentsum, linewidth = 1.5, color = 'blue', linestyle = ':', label = 'Incoherent sum')
ax2.set_xlabel('Delay time')
ax2.set_ylabel('Average Intensity')
ax2.axes.xaxis.set_ticks([])
ax2.axes.yaxis.set_ticks([])
ax2.legend(prop={'size': 8})
ax2.set_title('Detector response')
fig.tight_layout()


fig2 =  plt.figure(figsize = (1,1))
ax3 = fig2.add_subplot(1,1,1)
x = np.linspace(0,1, int((N+1)/2)+1)
y = np.abs(np.fft.rfft(sig))
ax3.plot(x, y, linewidth = 1, color = 'black')
plt.xlim(0, 0.05)
ax3.axes.xaxis.set_ticks([])
ax3.axes.yaxis.set_ticks([])
ax3.set_xlabel('Frequency',fontdict={'fontsize': 8})
ax3.set_ylabel('Power',fontdict={'fontsize': 8})